In [1]:
import os

#if not os.path.exists("train.csv"):
from datasets import *
dset = load_dataset("dair-ai/emotion")

# code from https://stackoverflow.com/a/76218276
train_testvalid = dset['train'].train_test_split(test_size=0.2)
# Split the 10% test + valid in half test, half valid
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)
# gather everyone if you want to have a single DatasetDict
#dset = DatasetDict({
 #   'train': train_testvalid['train'],
  #  'test': test_valid['test'],
   # 'valid': test_valid['train']})
dset
#else:
 #   print("file exists, skipping")

/home/codespace/.python/current/lib/python3.10/site-packages/datasets/load.py:1461: FutureWarning: The repository for dair-ai/emotion contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/dair-ai/emotion
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

## Cleaning Function to clean the dataset text

In [2]:
import spacy
import pandas as pd
import numpy as np
import re
#if not os.path.exists("test.csv"):
    # Check if the spaCy model is loaded, otherwise install it
try:
    nlp = spacy.load("en_core_web_sm")
except OSError:
    print("Downloading the 'en_core_web_sm' model")
    !python -m spacy download en_core_web_sm
    nlp = spacy.load("en_core_web_sm")

train_df = pd.DataFrame(dset["train"])
test_df = pd.DataFrame(dset['test'])
val_df = pd.DataFrame(dset['validation'])
i = 0
def cleaning(text):
    doc = nlp(text.lower().strip())
    cleaned_tokens = []
    global i
    print(f"Cleaning: {i}")
    for token in doc:
        if not token.is_stop and not token.is_punct and not token.is_space:
            lemma = re.sub(r'\W', '', token.lemma_)
            if lemma:
                cleaned_tokens.append(lemma)
    i += 1
    return ' '.join(cleaned_tokens)

# Apply the cleaning function to the text column
train_df["cleaned_text"] = train_df["text"].apply(cleaning)
test_df["cleaned_text"] = test_df["text"].apply(cleaning)
val_df["cleaned_text"] = val_df["text"].apply(cleaning)

print(train_df[['text', 'cleaned_text']].head())  # Display the original and cleaned text for verification

train_df.to_csv("train.csv")
test_df.to_csv("test.csv")
val_df.to_csv("val.csv")

#else: 
   # print("file exists, skipping")
    #train_df = pd.read_csv("train.csv")
    #test_df = pd.read_csv("test.csv")
    #val_df = pd.read_csv("val.csv")    

Cleaning: 0
Cleaning: 1
Cleaning: 2
Cleaning: 3
Cleaning: 4
Cleaning: 5
Cleaning: 6
Cleaning: 7
Cleaning: 8
Cleaning: 9
Cleaning: 10
Cleaning: 11
Cleaning: 12
Cleaning: 13
Cleaning: 14
Cleaning: 15
Cleaning: 16
Cleaning: 17
Cleaning: 18
Cleaning: 19
Cleaning: 20
Cleaning: 21
Cleaning: 22
Cleaning: 23
Cleaning: 24
Cleaning: 25
Cleaning: 26
Cleaning: 27
Cleaning: 28
Cleaning: 29
Cleaning: 30
Cleaning: 31
Cleaning: 32
Cleaning: 33
Cleaning: 34
Cleaning: 35
Cleaning: 36
Cleaning: 37
Cleaning: 38
Cleaning: 39
Cleaning: 40
Cleaning: 41
Cleaning: 42
Cleaning: 43
Cleaning: 44
Cleaning: 45
Cleaning: 46
Cleaning: 47
Cleaning: 48
Cleaning: 49
Cleaning: 50
Cleaning: 51
Cleaning: 52
Cleaning: 53
Cleaning: 54
Cleaning: 55
Cleaning: 56
Cleaning: 57
Cleaning: 58
Cleaning: 59
Cleaning: 60
Cleaning: 61
Cleaning: 62
Cleaning: 63
Cleaning: 64
Cleaning: 65
Cleaning: 66
Cleaning: 67
Cleaning: 68
Cleaning: 69
Cleaning: 70
Cleaning: 71
Cleaning: 72
Cleaning: 73
Cleaning: 74
Cleaning: 75
Cleaning: 76
Cleaning:

In [3]:
# This is to remove the 5 from the dfs
train_df = train_df[train_df['label'] != 5]
test_df = test_df[test_df['label'] != 5]
val_df = val_df[val_df['label'] != 5]

train_df = train_df.dropna(subset=["cleaned_text"])
test_df = test_df.dropna(subset=["cleaned_text"])
val_df = val_df.dropna(subset=["cleaned_text"])

#### 4 Different types of embeddings

In [4]:
from sklearn.feature_extraction.text import CountVectorizer
def create_bag_of_words(dataframe):
    vectorizer = CountVectorizer()
    X_bow = vectorizer.fit_transform(dataframe['cleaned_text'])
    return X_bow, vectorizer.get_feature_names_out()

X_bow, features_bow = create_bag_of_words(train_df)
X_test_bow, features_test_bow = create_bag_of_words(test_df)
print("Bag of Words features:", X_bow.shape)

Bag of Words features: (15428, 11592)


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
def create_tfidf_features(dataframe):
    vectorizer = TfidfVectorizer()
    X_tfidf = vectorizer.fit_transform(dataframe['cleaned_text'])
    return X_tfidf, vectorizer.get_feature_names_out()

X_tfidf, features_tfidf = create_tfidf_features(train_df)
X_test_tfidf, features_test_tfidf = create_tfidf_features(test_df)
print("TF-IDF features:", X_tfidf.shape)


TF-IDF features: (15428, 11592)


In [6]:
from gensim.models import Word2Vec
import numpy as np

def create_word2vec_embeddings(dataframe):
    sentences = [text.split() for text in dataframe['cleaned_text']]
    model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
    word_vectors = model.wv

    # Average Word Vectors for each text
    def document_vector(doc):
        return np.mean([word_vectors[w] for w in doc if w in word_vectors], axis=0)

    X_w2v = np.array([document_vector(text) for text in sentences if document_vector(text).shape != ()])
    return X_w2v


X_w2v = create_word2vec_embeddings(train_df)
X_test_w2v = create_word2vec_embeddings(test_df)
print("Word2Vec features shape:", X_w2v.shape)

Word2Vec features shape: (15428, 100)


In [7]:
from sentence_transformers import SentenceTransformer
#if not os.path.exists('X_bert.npy'):
def create_bert_embeddings(dataframe):
    model = SentenceTransformer('all-MiniLM-L6-v2')
    X_bert = model.encode(dataframe['cleaned_text'], show_progress_bar=True)
    return X_bert

X_bert = create_bert_embeddings(train_df)
X_test_bert = create_bert_embeddings(test_df)
np.save('X_bert.npy', X_bert)
np.save('X_test_bert.npy', X_test_bert)

#else:    
#    print("file exists, skipping")
#    X_bert = np.load('X_bert.npy')
#    X_test_bert = np.load('X_test_bert.npy') 

print("BERT Embeddings shape:", X_bert.shape)

KeyError: 7122

In [ ]:
train_df.loc[7122]

KeyError: 7122

In [ ]:
y_train = (train_df['label'])
y_test = (test_df['label'])
y_val = val_df['label']

In [ ]:
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_bert, y_train)
y_pred_test = clf.predict(X_test_bert)
print(classification_report(y_test, y_pred_test))

ValueError: Found input variables with inconsistent numbers of samples: [16000, 15428]

In [ ]:
clf2 = LogisticRegression(penalty=None)
clf2.fit(X_bert, y_train)
y_pred_test2 = clf2.predict(X_test_bert)
print(classification_report(y_test, y_pred_test2))

              precision    recall  f1-score   support

           0       0.72      0.73      0.73       581
           1       0.74      0.79      0.77       695
           2       0.55      0.40      0.47       159
           3       0.62      0.57      0.60       275
           4       0.60      0.66      0.63       224
           5       0.62      0.48      0.54        66

    accuracy                           0.69      2000
   macro avg       0.64      0.61      0.62      2000
weighted avg       0.68      0.69      0.68      2000



/home/codespace/.local/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
